In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import matplotlib
matplotlib.use('Agg')
#%matplotlib inline
import matplotlib.pyplot as plt

phase_list=["A1","A2","A31","A32","A41","A42","A5","A6","A7"]

mouse=6

for ph in range(8,len(phase_list)):

    phase=phase_list[ph]

    inputXY='ID138464-%s_%s_XY.txt' %(mouse,phase)

    dfx = pd.read_table(inputXY,header=0)

    inputT='ID138464-%s_long_trace_%s.csv'%(mouse,phase)

    dfl=pd.read_csv(inputT,header=0)

    dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))

    for i in range(2,int(dfx.shape[0])):

        for j in range(0,10):

            for k in range(0,len(dfx_m.columns)):

                dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]

    df=pd.concat([dfx_m, dfl], axis=1)
    df=df.dropna(how='any')

    Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)

    df_mean=pd.DataFrame(index=Index,columns=[])

    for i in range(0,len(df.columns)):

        df_mean[df.columns[i]]=0

    for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):

        i = i_n - int(df.iloc[0].loc['Slice'])

        df_mean.iloc[i].loc['Slice']=i_n

        dfs = df[df['Slice']==i_n]

        for k in range(0,df.columns.get_loc('Time2')+1):

            df_mean.iloc[i,k]=dfs.iloc[0,k]


        for j in range(dfs.columns.get_loc(' C000'),dfs.columns.get_loc(' C261')+1):

            df_mean.iloc[i,j]=dfs[dfs.columns[j]].mean()

    dfmean=df_mean.diff()
    dfmean.loc[:,:'Time2']=df_mean.loc[:,:'Time2']
    dfmean=dfmean.dropna(how='any')


    Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']),1)

    df_max=pd.DataFrame(index=Index,columns=[])

    for i in range(0,len(df.columns)):

        df_max[df.columns[i]]=0

    for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-11].loc['Slice']+1)):

        i = i_n - int(df.iloc[0].loc['Slice'])

        df_max.iloc[i].loc['Slice']=i_n

        dfs = df[df['Slice']==i_n]

        for k in range(0,df.columns.get_loc('Time2')+1):

            df_max.iloc[i,k]=dfs.iloc[0,k]


        for j in range(dfs.columns.get_loc(' C000'),dfs.columns.get_loc(' C261')+1):

            df_max.iloc[i,j]=dfs[dfs.columns[j]].max()

    dfmax=df_max.diff()
    dfmax.loc[:,:'Time2']=df_max.loc[:,:'Time2']
    dfmax=dfmax.dropna(how='any')

    dfmax_std=pd.DataFrame(columns=dfmax.columns,index=dfmax.index)
    for k in range(dfmax_std.columns.get_loc('Slice'),dfmax_std.columns.get_loc(' C000')):
        dfmax_std[dfmax_std.columns[k]]=dfmax[dfmax_std.columns[k]]
    for i in range(dfmax_std.columns.get_loc(' C000'),dfmax_std.columns.get_loc(' C261')+1):
        for j in range(0,dfmax_std.shape[0]):
            dfmax_std.iloc[j,i]=((dfmax.iloc[j,i]-dfmax[dfmax_std.columns[i]].mean())/dfmax[dfmax_std.columns[i]].std())

    dfmean_std=pd.DataFrame(columns=dfmean.columns,index=dfmean.index)
    for k in range(dfmean_std.columns.get_loc('Slice'),dfmean_std.columns.get_loc(' C000')):
        dfmean_std[dfmean_std.columns[k]]=dfmean[dfmean_std.columns[k]]
    for i in range(dfmean_std.columns.get_loc(' C000'),dfmean_std.columns.get_loc(' C261')+1):
        for j in range(0,dfmean_std.shape[0]):
            dfmean_std.iloc[j,i]=((dfmean.iloc[j,i]-dfmean[dfmean_std.columns[i]].mean())/dfmean[dfmean_std.columns[i]].std())

    for j in range(dfmean.columns.get_loc(' C000'),dfmean.columns.get_loc(' C261')+1):

        plt.plot(dfmean_std['Time1'],dfmean_std[dfmean.columns[j]],color='black')

        #plt.plot(dfmax_std['Time1'],dfmax_std[dfmax.columns[j]],color='black')

        #plt.ylim=(-5.0,5.0)

        for i in range(0,dfmean.shape[0]):

            if dfmean.iloc[i].loc['FZ'] == 1:

                if dfmean.iloc[i].loc['X']  <= 125:

                    if dfmean.iloc[i].loc['Y']  <=100:

                        Color='lightpink'

                    else:

                        Color='lightblue'

                else:

                    if dfmean.iloc[i].loc['Y']  <=100:

                        Color='silver'

                    else:

                        Color='lightgreen'


                plt.fill([dfmean.iloc[i].loc['Time1'],dfmean.iloc[i].loc['Time1']+1,dfmean.iloc[i].loc['Time1']+1,dfmean.iloc[i].loc['Time1']],[-5.0,-5.0,10.0,10.0],color=Color)

            if (dfmean.iloc[i].loc['Shock'] == 1):

                plt.plot([dfmean.iloc[i].loc['Time1'],dfmean.iloc[i].loc['Time1']],[-5.0,10.0],color='red',linestyle='dashed')

            plt.plot([dfmean.iloc[0].loc['Time1'],dfmean.iloc[dfmean.shape[0]-1].loc['Time1']],[5.0,5.0],color='red',linewidth=0.5)
            plt.plot([dfmean.iloc[0].loc['Time1'],dfmean.iloc[dfmean.shape[0]-1].loc['Time1']],[4.0,4.0],color='red',linewidth=0.5)
            plt.plot([dfmean.iloc[0].loc['Time1'],dfmean.iloc[dfmean.shape[0]-1].loc['Time1']],[3.0,3.0],color='red',linewidth=0.5)
            plt.plot([dfmean.iloc[0].loc['Time1'],dfmean.iloc[dfmean.shape[0]-1].loc['Time1']],[2.0,2.0],color='red',linewidth=0.5)
            plt.plot([dfmean.iloc[0].loc['Time1'],dfmean.iloc[dfmean.shape[0]-1].loc['Time1']],[1.0,1.0],color='red',linewidth=0.5)


        plt.ylim=(-5.0,10.0)
        plt.xlabel('Time1')
        plt.ylabel('Ca2+Img(std)')    
        title_name='fig2/slicediffini_phase%s_mouse%s_%s' % (phase,mouse,dfmean.columns[j])
        plt.title(title_name)
        file_name=title_name+'.png'
        plt.savefig(file_name)
        plt.figure(figsize=(5.4, 3.6), dpi=80)
        plt.show()

/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rentak

In [2]:
for j in range(df.columns.get_loc(' C219'),df.columns.get_loc(' C261')+1):   

    df_m=df[(df['mconseq_0']<0)]

    df_p=df[(df['mconseq_0']>=0) & (df['valid_0']==1)]

    plt.hist([df_m[df.columns[j]],df[df.columns[j]],df_p[df.columns[j]]],bins=10,normed=True,label=['nonFZ','ALL','FZ'])
    #plt.hist(df_m[df.columns[j]])
    plt.xlabel('Ca2+img')
    plt.ylabel('density')

    title_name='fig1/histdiffini_phase%s_mouse%s_%s' % (phase,mouse,dfmean.columns[j])
    plt.title(title_name)
    file_name=title_name+'.png'
    #plt.savefig(file_name)
    #plt.figure(figsize=(5.4, 3.6), dpi=80)
    plt.show()

KeyError: 'mconseq_0'